# Processing to have final data 
After annotate the meta data of census and have the samples matching
Note, now need to process the census again to have the distribution of HH correctly, maybe remove hhinc as well. Also include the negative hhinc

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

from PopSynthesis.Generator_data.generate_combine_census.utils import *

In [2]:
# Import the meta data
meta_df_hh = pd.read_csv("./processed_data/meta_data_hh_manual.csv")
meta_df_pp = pd.read_csv("./processed_data/meta_data_pp_manual.csv")

In [3]:
geo_lev = "POA"
data_raw = process_from_census_data(geo_lev=geo_lev)

In [4]:
persons_cols = [x for x in data_raw.columns if "Person_" in x]
hh_cols = [x for x in data_raw.columns if "Dwelling_" in x]

pp_marg_raw = data_raw[persons_cols]
hh_marg_raw = data_raw[hh_cols]

In [5]:
cols_have_geo_pp = [x for x in pp_marg_raw.columns if geo_lev in x]
assert len(cols_have_geo_pp) == 1
tot_pp_seri = pp_marg_raw[cols_have_geo_pp[0]]

In [6]:
ls_cols = [x for x in hh_marg_raw.columns if "Dwelling_NPRD Number of Persons Usually Resident in Dwelling" in x and "Not applicable" not in x]
tot_hh_seri = hh_marg_raw[ls_cols].sum(axis=1) + 55

In [7]:
def process_data_all(type_att="Dwelling"):
    if type_att == "Dwelling":
        meta_data, raw_marg_data, tot_seri = meta_df_hh, hh_marg_raw, tot_hh_seri
    elif type_att == "Person":
        meta_data, raw_marg_data, tot_seri = meta_df_pp, pp_marg_raw, tot_pp_seri
    else:
        raise ValueError("No exist this type")
    
    ls_df_to_concat = []
    for att_code in meta_data["att_code_census"].unique():
        dict_hold_val = {}

        sub_df = meta_data[meta_data["att_code_census"]==att_code]
        assert len(sub_df["att_description"].unique()) == 1
        assert len(sub_df["att_sample"].unique()) == 1
        
        name_full = type_att + "_" + att_code + " " + sub_df["att_description"].unique()[0]
        att_sample = sub_df["att_sample"].unique()[0]
        all_state_census = sub_df["state_census"].unique()
        all_cols_name = [f"{name_full}__{state_census}" for state_census in all_state_census]
        gb_df = sub_df.groupby("state_sample")["state_census"].apply(lambda x: list(x))

        # Process delete all and shift the value to others cols
        re_df = raw_marg_data.fillna(0)
        if "SHIFT" in gb_df.index:
            sub_del_name = gb_df["SHIFT"]
            cols_del = [f"{name_full}__{del_census}" for del_census in sub_del_name]
            ls_not_del = list(set(all_cols_name) - set(cols_del))
            to_plus_seri = raw_marg_data[cols_del].sum(axis=1) / len(ls_not_del)
            re_df = raw_marg_data[ls_not_del].add(to_plus_seri, axis="index")

        # Process the break when we want the value to shared between specific cols
        for att_sample_state in gb_df.index:
            if "BREAK" in att_sample_state:
                ls_cols_to_take = att_sample_state.split(":")[1].split("|")
                hold_cols = []
                for x in ls_cols_to_take:
                     hold_cols += gb_df[x]
                col_del = f"{name_full}__{gb_df[att_sample_state][0]}"
                cols_to_update = [f"{name_full}__{col_take}" for col_take in hold_cols]
                sub_df_of_this = re_df[cols_to_update].add((re_df[col_del] / len(ls_cols_to_take)), axis="index")
                # Update the final df
                re_df[cols_to_update] = sub_df_of_this

        for att_sample_state in gb_df.index:
            if "BREAK" not in att_sample_state and "DELETE" != att_sample_state and "SHIFT" != att_sample_state:
                ls_of_census = [f"{name_full}__{census_val}" for census_val in gb_df[att_sample_state]]
                final_seri_val = re_df[ls_of_census].sum(axis=1)
                dict_hold_val[(att_sample, att_sample_state)] = final_seri_val
        df_of_att = pd.DataFrame(dict_hold_val)
        # Process to intergalised the re_df
        after_inter = update_int_all(df_of_att, tot_seri)
        ls_df_to_concat.append(after_inter)

    final_df = pd.concat(ls_df_to_concat, axis=1)
    final_df.insert(loc=0, column=("zone_id", None), value=final_df.index)
    if type_att == "Dwelling":
        final_df.insert(loc=1, column=("sample_geog", None), value=2)
    return final_df

In [8]:
final_df_census_hh = process_data_all(type_att="Dwelling")
final_df_census_pp = process_data_all(type_att="Person")

In [9]:
a = final_df_census_hh[final_df_census_hh.columns[final_df_census_hh.columns.get_level_values(0)=="hhsize"]]
tem_s = 0
for i in range(1, 8):
    tem_s += a[("hhsize", str(i))] * i
tem_s += a[("hhsize", "8+")] * 10
tem_s.sum()

6464884

In [10]:
tot_pp_seri.sum()

6450747.0

In [36]:
a = final_df_census_hh.astype(int) < 0
dict_to_process = {}
for i, r in a.iterrows():
    if r.any():
        loc_cols = r[r].index
        dict_to_process[i] = list(loc_cols)

for idx, ls_cols in dict_to_process.items():
    print(idx, ls_cols)

In [16]:
# final_df_census_hh.to_csv("./hh_marginals_ipu.csv", index=False)
# final_df_census_pp.to_csv("./person_marginals_ipu.csv", index=False)

# Convert all to percentages

In [40]:
for col in final_df_census_hh.columns:
    if col[0] not in ["zone_id", "sample_geog", "Vehicle"]:
        final_df_census_hh[col] = final_df_census_hh[col] / tot_hh_seri

In [42]:
for col in final_df_census_pp.columns:
    if col[0] not in ["zone_id", "sample_geog", "Vehicle"]:
        final_df_census_pp[col] = final_df_census_pp[col] / tot_pp_seri

In [13]:
# Now we need to add special case of fueltype
veh_cols = [x for x in data_raw.columns if "Vehicle_" in x]
fuel_data = data_raw[veh_cols]
fuel_data = fuel_data.astype(int)
# fuel_data["total"] = fuel_data.sum(axis=1)
eletric_percen = fuel_data["Vehicle_Fuel type__Electric"] / fuel_data.sum(axis=1)

In [14]:
final_df_census_hh[("Vehicle", "Electric_ratio")] = eletric_percen
final_df_census_hh

zone_id sample_geog hhsize                                   ...  \
         NaN         NaN      1     2     3    4    5   6  7  8+  ...   
3000    3000           2  10393  8524  2026  656  118  27  4  58  ...   
3002    3002           2   1035  1019   190  101   38   9  0  55  ...   
3003    3003           2   1504  1591   476  223   40  11  7  55  ...   
3004    3004           2   2566  2382   511  227   33   6  5  55  ...   
3006    3006           2   5113  5033  1280  414   59  13  8  58  ...   
...      ...         ...    ...   ...   ...  ...  ...  .. ..  ..  ...   
3990    3990           2      6    20     8    4    3   0  0  55  ...   
3991    3991           2     37    65    16   19   10   9  0  55  ...   
3992    3992           2    134   201    72   60   24   9  0  55  ...   
3995    3995           2   1663  1834   629  451  228  73  8  58  ...   
3996    3996           2    806  1176   251  299  137  31  6  59  ...   

     totalvehs                              owndwell                           \
             0     1     2    3   4+ Being Purchased Being Rented Fully Owned   
3000     14716  5422   850  410  408            2450        16605        2280   
3002       467  1272   547   94   67             436         1295         675   
3003      1391  1936   427   97   56             797         2548         463   
3004      1211  3433   885  160   96            1209         3210        1268   
3006      4748  5839  1036  202  153            2140         8097        1529   
...        ...   ...   ...  ...  ...             ...          ...         ...   
3990        13    13    25   24   21              16            0          22   
3991        22    45    67   40   37              51           17          67   
3992        12   153   218   93   79             229           55         192   
3995       370  1950  1695  591  338            1326         1150        2316   
3996        90  1039  1208  300  128             687          494        1528   

                           Vehicle  
     Something Else Electric_ratio  
3000            471       0.004647  
3002             41       0.005305  
3003             99       0.004023  
3004             98       0.006074  
3006            212       0.003229  
...             ...            ...  
3990             58       0.000000  
3991             76       0.000000  
3992             79       0.000000  
3995            152       0.001053  
3996             56       0.001210  

[691 rows x 47 columns]

In [15]:
final_df_census_pp[("Vehicle", "Electric_ratio")] = eletric_percen
final_df_census_pp

zone_id   age                                                   ...  \
         NaN   0-9 10-19 100+  20-29  30-39 40-49 50-59 60-69 70-79  ...   
3000    3000  1131  2057    0  21588  10739  3433  2059  1275   619  ...   
3002    3002   210   179    0    840   1032   571   614   683   537  ...   
3003    3003   430   380    0   2832   2337   914   565   307   161  ...   
3004    3004   604   407    9   2432   2928  1530  1111  1048   953  ...   
3006    3006  1107   709    0   7932   7118  2452  1454  1108   612  ...   
...      ...   ...   ...  ...    ...    ...   ...   ...   ...   ...  ...   
3990    3990     3     6    0      7     13    12    25    25    25  ...   
3991    3991    28    48    0     40     42    41    67    62    49  ...   
3992    3992   146   144    0    130    156   164   164   232   106  ...   
3995    3995  1054  1201    4   1047   1165  1175  1497  1926  1571  ...   
3996    3996   656   660    3    366    598   650   774  1268  1070  ...   

           persinc                                                            \
     $600-799 p.w. $800-999 p.w. Missing/Refused Negative Income Zero Income   
3000          4688          3585            5585             227        7764   
3002           330           239             621              24         686   
3003           714           600            1183              29         621   
3004          1109           699            1967              36         737   
3006          1933          1709            2693              73        2119   
...            ...           ...             ...             ...         ...   
3990            15             5              22               3           9   
3991            31            28             107               6          24   
3992           134            99             311              10          55   
3995          1406           780            2456              72         555   
3996           718           430            1402              35         315   

        sex        anywork               Vehicle  
          F      M       N      Y Electric_ratio  
3000  21539  24113   15851  29801       0.004647  
3002   2545   2837    1849   3533       0.005305  
3003   3907   4069    2582   5394       0.004023  
3004   5827   6478    4567   7738       0.006074  
3006  11197  11941    6568  16570       0.003229  
...     ...    ...     ...    ...            ...  
3990     62     54      45     71       0.000000  
3991    192    196     203    185       0.000000  
3992    624    647     600    671       0.000000  
3995   5957   5536    6681   4812       0.001053  
3996   3408   3153    3839   2722       0.001210  

[691 rows x 30 columns]

In [43]:
final_df_census_hh.to_csv("./hh_marginals_percen.csv", index=False)
final_df_census_pp.to_csv("./person_marginals_percen.csv", index=False)

## Process the sample data now

In [17]:
# ori_sample_hh = pd.read_csv("./processed_data/ori_sample_hh.csv")
# ori_sample_pp = pd.read_csv("./processed_data/ori_sample_pp.csv")

In [18]:
ipu_sample_hh = ori_sample_hh.drop(columns=["Spouse","Child","Others","Grandchild","_weight"]).rename(columns={"hhid": "serialno"})
ipu_sample_pp = ori_sample_pp.drop(columns=["persid", "_weight"]).rename(columns={"hhid": "serialno"})
ipu_sample_hh.insert(loc=1, column="sample_geog", value=2)
ipu_sample_pp.insert(loc=1, column="sample_geog", value=2)

NameError: name 'ori_sample_hh' is not defined

In [ ]:
# ipu_sample_pp.to_csv("./pp_sample_ipu.csv", index=False)
# ipu_sample_hh.to_csv("./hh_sample_ipu.csv", index=False)